# Mutex Watershed

Use the `elf.segmentation` module for boundary based segmentation with the mutex watershed algorithm: [The Mutex Watershed: Efficent, Parameter-Free Image Partitionong](http://openaccess.thecvf.com/content_ECCV_2018/html/Steffen_Wolf_The_Mutex_Watershed_ECCV_2018_paper.html).
We use data from the paper based on the [ISBI 2012 EM Segmentation challenge](http://brainiac2.mit.edu/isbi_challenge/home).
You can obtain this data [here](https://hcicloud.iwr.uni-heidelberg.de/index.php/s/6LuE7nxBN3EFRtL).

The mutex watershed can operate directly on pixel affinity maps.
It produces a segmentation by partitioning the grid graph, taking into acount long range pixel connections. This is achieved by greedily connecting pixels that are joined by a path of local affinity edges **unless** there exists a long range edge that prevents this join.

In addition to the default elf dependencies, you will need to install [affogato](https://github.com/constantinpape/affogato) to run this example.

## Preparation

In [ ]:
%gui qt5 
import numpy as np

# import napari for data visualisation
import napari

# import the segmentation functionality from elf
import elf.segmentation.mutex_watershed as mws

# import the open_file function from elf, which supports opening files
# in hdf5, zarr, n5 or knossos file format
from elf.io import open_file

In [ ]:
# read the data
# you can download the example data from here:
# https://hcicloud.iwr.uni-heidelberg.de/index.php/s/6LuE7nxBN3EFRtL
data_path = '/home/pape/Work/data/isbi/isbi_test_volume.h5'  # adjust this path
with open_file(data_path, 'r') as f:
    # load the raw data
    raw = f['raw'][:]
    # load the affinities
    affs = f['affinities'][:]

## Segment via mutex watershed

In [ ]:
# set all parameters for the mutex watershed

# the offsets corresponds the pixel transitions defied by the inidvidual affinity channels
# for example, [-1, 0, 0] means that the affinity channel encodes the transition to
# the pixel one z slice below.
# [0, -9, 0] mean that the affinity channel encode the transition to 9 pixels to the left
offsets = [[-1, 0, 0], [0, -1, 0], [0, 0, -1],
           [-1, -1, -1], [-1, 1, 1], [-1, -1, 1], [-1, 1, -1],
           [0, -9, 0], [0, 0, -9],
           [0, -9, -9], [0, 9, -9], [0, -9, -4], [0, -4, -9], [0, 4, -9], [0, 9, -4],
           [0, -27, 0], [0, 0, -27]]

# The strides are used to sub-sample the long range edges, which are used for repulsive
# connections in the mutex watershed.
# This reduces the runtime and is ok, because we have more long range then local affinity channels.
strides = [1, 10, 10]

# if randomize_strides is True, the sub-sampling of long-range edges is done at random.
# this usually improves resutls by avoiding sampling artefacts, but it makes the result
# not fully reproducible
randomize_strides = True

In [ ]:
# run the algorithm
segmentation = mws.mutex_watershed(affs, offsets, strides,
                                   randomize_strides=True)

In [ ]:
# visualize the results
# TODO switch to new napari syntax
# napari.view_image(raw, name='raw')
# napari.view_image(affs, name='affinities')
# napari.add_labels(segmentation, name='mws-segmentation')

viewer = napari.Viewer()
viewer.add_image(raw, name='raw')
viewer.add_image(affs, name='affinities')
viewer.add_labels(segmentation, name='mws-segmentation')

## Block-wise MWS

There's also a block-wise implementation of the mutex watershed (that uses Multicut to stitch block results).
You can use it to segment larger volumes, where normal mutex watershed takes too long to run.

In [ ]:
# NOTE due to an issue with the current mws implementation, please
# reload the affinities before running the blockwise segmentation
block_shape = [10, 256, 256]
blockwise_seg = mws.blockwise_mutex_watershed(affs, offsets, strides,
                                              block_shape, randomize_strides=True)
print(blockwise_seg.shape)

In [ ]:
# visualize the results
# TODO switch to new napari syntax
# napari.view_image(raw, name='raw')
# napari.view_image(affs, name='affinities')
# napari.add_labels(segmentation, name='mws-segmentation')

viewer = napari.Viewer()
viewer.add_image(raw, name='raw')
viewer.add_image(affs, name='affinities')
viewer.add_labels(segmentation, name='mws-segmentation')
viewer.add_labels(blockwise_seg, name='blockwise-segmentation')